In [2]:
import pyhealth
import pandas as pd
from typing import *
import numpy as np
from pyhealth.models import Transformer, RNN
from itertools import chain
import random
import torch
from sklearn.model_selection import train_test_split


In [3]:
radiolung_df = pd.read_excel("20230713_BDMetaData_CanRuti.xlsx")

In [4]:
def collate_fn_dict(batch):
    return {key: [d[key] for d in batch] for key in batch[0]}



class RadioLungDataset():
    
    def __init__(self, path_df: str):
        
        self.df = pd.read_excel(path_df)
        self.input_info = {} # com estan cada feature_key
        self.map_feature_token_space()

        
    def __len__(self):
        return self.df.shape[0]
    
    def map_feature_token_space(self) -> None:
        feature_keys = self.df.columns
        for feature_key in feature_keys:
           
            token_space = self.df[feature_key].unique().tolist() #token_space = self.df[feature_key].unique().flatten().tolist()
            if pd.api.types.is_string_dtype(self.df[feature_key] ) == True:
                self.input_info[feature_key] = {"type": str, "dim": 2} 
            else:
                self.input_info[feature_key] = {"type": int, "dim": 2, 'len':1}# ha petat i tb i he ficat len 
             
                self.df[feature_key]= (self.df[feature_key].apply(lambda x: [x])) # he ficat tots es num dins una llista pq vol aquest format
            
           #todo: modificar pq puguis ficar aquesta linia enlloc de int/float -->  (type(self.df[feature_key].iloc[0]).__name__)


    def case1_case2_binary(self): #s'han de natejar dades crec
        feature_bin = ["air_pollution", "ph_cancer", "COPD", "ASTHMA", "bronchiectasis", "TB", "pneumonia"]
        
        dic_bi = {}
        for i, feature in enumerate(feature_bin):
            dic_bi[feature] = {0: i * 2, 1: i * 2 + 1}
        
        for feature in feature_bin:
            self.df[feature] = self.df[feature].map(dic_bi[feature])
        print(self.df['TB'])
        return dic_bi
      
         
    def get_all_tokens(self, key:str):
        return self.df[key].unique().flatten().tolist()
    
    

    def __getitem__(self, idx):
        sample = self.df.iloc[idx]
        sample_return = {fkey: [value] if fkey != "type" else value for fkey, value in sample.items()}
        sample_return.update({"patient_id": sample_return["id"]})
        sample_return.pop("id")
        return sample_return

In [5]:
dataset = RadioLungDataset(path_df="20230713_BDMetaData_CanRuti.xlsx")


In [6]:
dataset[3]
dataset.input_info
dataset.get_all_tokens(key="type")


['Malignant', 'Benign', nan]

In [7]:
model = Transformer(dataset=dataset, 
                    feature_keys=["sex", "BMI",'lobe','global_location','histological_diagnosis'],
                    label_key="type", # imagin que es lo que s'ha de predir
                    mode="multiclass", # no hi ha nateja de dataset i posa que hi ha 3 tipus enlloc de ser binari
                    embedding_dim=128)

1 




In [8]:
def split_by_patient(
    dataset,
    ratios: Union[Tuple[float, float, float], List[float]],
    seed: Optional[int] = None,):

    if seed is not None:
        np.random.seed(seed)
    assert sum(ratios) == 1.0, "ratios must sum to 1.0"
    patient_ids = [dataset[i]['patient_id'][0] for i in range(len(dataset))]
    num_patients = len(patient_ids)
    
    idx = np.arange(num_patients)  

    train_index, temp_idx = train_test_split(idx, test_size=0.3, random_state=42)
    val_index, test_index = train_test_split(temp_idx, test_size=0.5, random_state=42)  

    train_dataset = torch.utils.data.Subset(dataset, train_index)
    val_dataset = torch.utils.data.Subset(dataset, val_index)
    test_dataset = torch.utils.data.Subset(dataset, test_index)
    return train_dataset, val_dataset, test_dataset



In [9]:
train_ds, val_ds, test_ds = split_by_patient(dataset, (0.8, 0.1, 0.1))


In [10]:
train_loader = get_dataloader(train_ds, batch_size=32, shuffle=True)
val_loader = get_dataloader(val_ds, batch_size=32, shuffle=False)
test_loader = get_dataloader(test_ds, batch_size=32, shuffle=False)

In [14]:

data_batch = next(iter(train_loader))
print(data_batch.keys())
print(data_batch["type"])



dict_keys(['hospital', 'birthdate', 'sex', 'education', 'BMI', 'smoking', 'packyear_index', 'air_pollution', 'fh_cancer', 'ph_cancer', 'COPD', 'ASTHMA', 'bronchiectasis', 'TB', 'pneumonia', 'FVC', 'FVCL', 'FEV1', 'FEV1L', 'indice', 'DLCO', 'id.1', 'patient_id', 'nodule_id', 'ct_id', 'ct_id_DB', 'petscan_id', 'surgery_id', 'date', 'type', 'lobe', 'global_location', 'xray', 'tsize', 'cdiff', 'necrosis', 'vinfiltration', 'histological_diagnosis', 'TNM', 'CD34', 'CD8', 'PDL1', 'WasItDone', 'IfDone', 'whichtest', 'whichtest2', 'id.2', 'hospital.1', 'date.1', 'hospital_ct', 'slice_thickness', 'nodule_diameter', 'nodule_shape', 'nodule_density', 'emphysema', 'contrast', 'lowdosect', 'CTFile'])
['Malignant', 'Malignant', 'Malignant', 'Benign', 'Malignant', 'Benign', 'Malignant', 'Malignant', 'Malignant', 'Benign', 'Malignant', 'Benign', nan, 'Malignant', 'Benign', 'Benign', 'Malignant', 'Malignant', 'Malignant', 'Malignant', 'Malignant', 'Malignant', 'Malignant', 'Malignant', 'Malignant', 'Ben

In [71]:
model = Transformer(dataset=dataset, 
                    feature_keys=["sex", "BMI",'lobe','global_location','histological_diagnosis'],
                    label_key="type", # imagin que es lo que s'ha de predir
                    mode="multiclass", # no hi ha nateja de dataset i posa que hi ha 3 tipus enlloc de ser binari
                    embedding_dim=128)

1 




In [ ]:
for idx, object in enumerate(train_loader)

In [74]:
from pyhealth.trainer import Trainer

trainer = Trainer(model=model)
trainer.train(
train_dataloader=train_loader,
val_dataloader=val_loader,
epochs=50,

  )

Transformer(
  (embeddings): ModuleDict(
    (sex): Embedding(4, 128, padding_idx=0)
    (lobe): Embedding(7, 128, padding_idx=0)
    (global_location): Embedding(6, 128, padding_idx=0)
    (histological_diagnosis): Embedding(13, 128, padding_idx=0)
  )
  (linear_layers): ModuleDict(
    (BMI): Linear(in_features=1, out_features=128, bias=True)
  )
  (transformer): ModuleDict(
    (sex): TransformerLayer(
      (transformer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadedAttention(
            (linear_layers): ModuleList(
              (0-2): 3 x Linear(in_features=128, out_features=128, bias=False)
            )
            (output_linear): Linear(in_features=128, out_features=128, bias=False)
            (attention): Attention()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (feed_forward): PositionwiseFeedForward(
            (w_1): Linear(in_features=128, out_features=512, bias=True)
            (w_2): Linear(in_features=51

Epoch 0 / 50:   0%|          | 0/3 [00:00<?, ?it/s]



712873817238



Epoch 0 / 50: 100%|██████████| 3/3 [00:02<00:00,  1.49it/s]

holhaestic mirant es batch 
 [[0.0]]


712873817238



712873817238



712873817238



712873817238

holhaestic mirant es batch 
 [[0.0]]


712873817238



712873817238



712873817238



712873817238

holhaestic mirant es batch 
 [[28.0]]


712873817238



712873817238



712873817238

--- Train epoch-0, step-3 ---
loss: 1.2279


...



712873817238

holhaestic mirant es batch 
 [[20.8]]


712873817238



712873817238



712873817238



712873817238

holhaestic mirant es batch 
 [[27.5]]


712873817238



712873817238



712873817238



712873817238

holhaestic mirant es batch 
 [[33.0]]


712873817238



712873817238



Epoch 49 / 50: 100%|██████████| 3/3 [00:00<00:00, 35.01it/s]



712873817238

--- Train epoch-49, step-150 ---
loss: 0.0001



Evaluation: 100%|██████████| 1/1 [00:00<00:00, 61.51it/s]



712873817238

holhaestic mirant es batch 
 [[27.0]]


712873817238



712873817238



712873817238

--- Eval epoch-49, step-150 ---
accuracy: 1.0000
f1_macro: 1.0000
f1_micro: 1.0000
loss: 0.0000


In [85]:
"""
 # for case 3: [[1.5, 2.0, 0.0], ...]
            elif (dim_ == 2) and (type_ in [float, int]):
                import pdb
                pdb.set_trace()
                x, mask = self.padding2d(kwargs[feature_key])
                # (patient, event, values)
                x = torch.tensor(x, dtype=torch.float, device=self.device)
                # (patient, event, embedding_dim)
                x = self.linear_layers[feature_key](x)
                # (patient, event)
                mask = mask.bool().to(self.device)
"""

# try the model
ret = model(**data_batch)
print(ret)

# try loss backward
ret["loss"].backward()






712873817238



KeyError: 'sex'

In [75]:
trainer.evaluate(test_loader)


Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]



712873817238



Evaluation: 100%|██████████| 1/1 [00:02<00:00,  2.05s/it]

holhaestic mirant es batch 
 [[24.4]]


712873817238



712873817238



712873817238



{'accuracy': 1.0,
 'f1_macro': 1.0,
 'f1_micro': 1.0,
 'loss': 0.00036061316495761275}

#### COSES QUE HE FET PERO QUE NO SERVEIXEN EN GENERAL PER RES

In [17]:
from pyhealth.tokenizer import Tokenizer
print(radiolung_df["sex"].unique())
token_space = radiolung_df[["fh_cancer"]].to_numpy().flatten().tolist()
print(token_space
      )

['Woman' ' Man']
[' No', ' No', ' No', ' No', ' No', ' No', ' No', ' (Yes) others', ' (Yes) others', ' No', ' No', ' No', ' No', ' No', ' No', ' No', ' (Yes) others', ' No', ' No', ' No', ' No', ' No', ' No', ' No', ' No', ' (Yes) others', ' No', ' No', ' No', ' No', '(Yes) Lung cancer', ' (Yes) others', ' No', '(Yes) Lung cancer', ' No', ' (Yes) others', '(Yes) Lung cancer', ' No', ' (Yes) others', ' No', ' (Yes) others', ' No', ' No', '(Yes) Lung cancer', ' (Yes) others', ' (Yes) others', ' No', ' No', ' No', ' (Yes) others', '(Yes) others', '(Yes) Lung cancer', nan, ' No', ' No', nan, '(Yes) others', '(Yes) Lung cancer', ' No', ' No', nan, ' No', ' No', '(Yes) others', '(Yes) Lung cancer', '(Yes) Lung cancer', ' No', ' No', '(Yes) others', ' No', ' No', '(Yes) others', nan, ' No', '(Yes) Lung cancer', '(Yes) others', ' No', ' No', nan, '(Yes) others', ' No', nan, nan, '(Yes) others', ' No', nan, '(Yes) others', nan, '(Yes) Lung cancer', ' No', ' No', ' No', '(Yes) others', ' No', '(

In [18]:
tokenizer = Tokenizer(tokens=token_space, special_tokens=["<pad>", "<unk>"])

In [19]:
tokenizer.vocabulary.token2idx

{'<pad>': 0,
 '<unk>': 1,
 ' No': 2,
 ' (Yes) others': 3,
 '(Yes) Lung cancer': 4,
 '(Yes) others': 5,
 nan: 6}

In [20]:
tokenizer.convert_tokens_to_indices(["hola", "pepe"])

[1, 1]

In [1]:
from pyhealth.datasets import SampleEHRDataset

samples = [
    {
        "patient_id": "patient-0",
        "visit_id": "visit-0",
        "single_vector": [1, 2, 3],
        "list_codes": ["505800458"],  # NDC
        "list_vectors": [[1.0, 2.55, 3.4], [4.1, 5.5, 6.0]],
        "list_list_codes": [["A05B", "A05C", "A06A"], ["A11D", "A11E"]],  # ATC-4
        "list_list_vectors": [
            [[1.8, 2.25, 3.41], [4.50, 5.9, 6.0]],
            [[7.7, 8.5, 9.4]],
        ],
        "label": 1,
    },
    {
        "patient_id": "patient-0",
        "visit_id": "visit-1",
        "single_vector": [1, 5, 8],
        "list_codes": [
            "55154191800",
            "551541928",
            "55154192800",
            "705182798",
            "70518279800",
        ],
        "list_vectors": [[1.4, 3.2, 3.5], [4.1, 5.9, 1.7], [4.5, 5.9, 1.7]],
        "list_list_codes": [["A04A", "B035", "C129"]],
        "list_list_vectors": [
            [[1.0, 2.8, 3.3], [4.9, 5.0, 6.6], [7.7, 8.4, 1.3], [7.7, 8.4, 1.3]],
        ],
        "label": 0,
    },
]

# dataset
dataset = SampleEHRDataset(samples=samples, dataset_name="test")

# data loader
from pyhealth.datasets import get_dataloader

train_loader = get_dataloader(dataset, batch_size=2, shuffle=True)

# model
model = Transformer(
    dataset=dataset,
    feature_keys=[
        "list_codes",
        "list_vectors",
        "list_list_codes",
        "list_list_vectors",
    ],
    label_key="label",
    mode="multiclass",
)

# data batch
data_batch = next(iter(train_loader))

print(data_batch)

# try the model
ret = model(**data_batch)
print(ret)

# try loss backward
ret["loss"].backward()

123123


NameError: name 'Transformer' is not defined

In [ ]:

class Attention(nn.Module):
    def forward(self, query, key, value, mask=None, dropout=None):
        scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(query.size(-1))
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)
        p_attn = torch.softmax(scores, dim=-1)
        if mask is not None:
            p_attn = p_attn.masked_fill(mask == 0, 0)
        if dropout is not None:
            p_attn = dropout(p_attn)
 
        return torch.matmul(p_attn, value), p_attn


class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0

        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
    
        self.linear_layers = nn.ModuleList(
            [nn.Linear(d_model, d_model, bias=False) for _ in range(3)]
        )
        self.output_linear = nn.Linear(d_model, d_model, bias=False)
        self.attention = Attention()

        self.dropout = nn.Dropout(p=dropout)

        self.attn_gradients = None
        self.attn_map = None

    # helper functions for interpretability
    def get_attn_map(self):
        return self.attn_map 
    
    def get_attn_grad(self):
        return self.attn_gradients

    def save_attn_grad(self, attn_grad):
        self.attn_gradients = attn_grad 

    # register_hook option allows us to save the gradients in backwarding
    def forward(self, query, key, value, mask=None, register_hook = False):
        batch_size = query.size(0)

        # 1) Do all the linear projections in batch from d_model => h x d_k
        query, key, value = [
            l(x).view(batch_size, -1, self.h, self.d_k).transpose(1, 2)
            for l, x in zip(self.linear_layers, (query, key, value))
        ]
        
        # 2) Apply attention on all the projected vectors in batch.
        if mask is not None:
            mask = mask.unsqueeze(1)
        x, attn = self.attention(query, key, value, mask=mask, dropout=self.dropout)
        
        self.attn_map = attn # save the attention map
        if register_hook:
            attn.register_hook(self.save_attn_grad)
        # 3) "Concat" using a view and apply a final linear.
        x = x.transpose(1, 2).contiguous().view(batch_size, -1, self.h * self.d_k)
  
        return self.output_linear(x)


class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)
        self.activation = nn.GELU()

    def forward(self, x, mask=None):
        x = self.w_2(self.dropout(self.activation(self.w_1(x))))
        if mask is not None:
            mask = mask.sum(dim=-1) > 0
            x[~mask] = 0
        return x


class SublayerConnection(nn.Module):
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = nn.LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        return x + self.dropout(sublayer(self.norm(x)))


class TransformerBlock(nn.Module):
    """Transformer block.

    MultiHeadedAttention + PositionwiseFeedForward + SublayerConnection

    Args:
        hidden: hidden size of transformer.
        attn_heads: head sizes of multi-head attention.
        dropout: dropout rate.
    """

    def __init__(self, hidden, attn_heads, dropout):
        super(TransformerBlock, self).__init__()
        self.attention = MultiHeadedAttention(h=attn_heads, d_model=hidden)
        self.feed_forward = PositionwiseFeedForward(
            d_model=hidden, d_ff=4 * hidden, dropout=dropout
        )
        self.input_sublayer = SublayerConnection(size=hidden, dropout=dropout)
        self.output_sublayer = SublayerConnection(size=hidden, dropout=dropout)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x, mask=None, register_hook = False):
        """Forward propagation.

        Args:
            x: [batch_size, seq_len, hidden]
            mask: [batch_size, seq_len, seq_len]

        Returns:
            A tensor of shape [batch_size, seq_len, hidden]
        """
        x = self.input_sublayer(x, lambda _x: self.attention(_x, _x, _x, mask=mask, register_hook=register_hook))
        x = self.output_sublayer(x, lambda _x: self.feed_forward(_x, mask=mask))
        return self.dropout(x)


class TransformerLayer(nn.Module):
    """Transformer layer.

    Paper: Ashish Vaswani et al. Attention is all you need. NIPS 2017.

    This layer is used in the Transformer model. But it can also be used
    as a standalone layer.

    Args:
        feature_size: the hidden feature size.
        heads: the number of attention heads. Default is 1.
        dropout: dropout rate. Default is 0.5.
        num_layers: number of transformer layers. Default is 1.
        register_hook: True to save gradients of attention layer, Default is False.
    Examples:
        >>> from pyhealth.models import TransformerLayer
        >>> input = torch.randn(3, 128, 64)  # [batch size, sequence len, feature_size]
        >>> layer = TransformerLayer(64)
        >>> emb, cls_emb = layer(input)
        >>> emb.shape
        torch.Size([3, 128, 64])
        >>> cls_emb.shape
        torch.Size([3, 64])
    """

    def __init__(self, feature_size, heads=1, dropout=0.5, num_layers=1):
        super(TransformerLayer, self).__init__()
        self.transformer = nn.ModuleList(
            [TransformerBlock(feature_size, heads, dropout) for _ in range(num_layers)]
        )

    def forward(
        self, x: torch.tensor, mask: Optional[torch.tensor] = None, register_hook = False
    ) -> Tuple[torch.tensor, torch.tensor]:
        """Forward propagation.

        Args:
            x: a tensor of shape [batch size, sequence len, feature_size].
            mask: an optional tensor of shape [batch size, sequence len], where
                1 indicates valid and 0 indicates invalid.

        Returns:
            emb: a tensor of shape [batch size, sequence len, feature_size],
                containing the output features for each time step.
            cls_emb: a tensor of shape [batch size, feature_size], containing
                the output features for the first time step.
        """
        if mask is not None:
            mask = torch.einsum("ab,ac->abc", mask, mask)
        for transformer in self.transformer:
            x = transformer(x, mask, register_hook)
        emb = x
        cls_emb = x[:, 0, :]
        return emb, cls_emb


class Transformer(BaseModel):
    """Transformer model.

    This model applies a separate Transformer layer for each feature, and then
    concatenates the final hidden states of each Transformer layer. The concatenated
    hidden states are then fed into a fully connected layer to make predictions.

    Note:
        We use separate Transformer layers for different feature_keys.
        Currentluy, we automatically support different input formats:
            - code based input (need to use the embedding table later)
            - float/int based value input
        We follow the current convention for the transformer model:
            - case 1. [code1, code2, code3, ...]
                - we will assume the code follows the order; our model will encode
                each code into a vector and apply transformer on the code level
            - case 2. [[code1, code2]] or [[code1, code2], [code3, code4, code5], ...]
                - we will assume the inner bracket follows the order; our model first
                use the embedding table to encode each code into a vector and then use
                average/mean pooling to get one vector for one inner bracket; then use
                transformer one the braket level
            - case 3. [[1.5, 2.0, 0.0]] or [[1.5, 2.0, 0.0], [8, 1.2, 4.5], ...]
                - this case only makes sense when each inner bracket has the same length;
                we assume each dimension has the same meaning; we run transformer directly
                on the inner bracket level, similar to case 1 after embedding table
            - case 4. [[[1.5, 2.0, 0.0]]] or [[[1.5, 2.0, 0.0], [8, 1.2, 4.5]], ...]
                - this case only makes sense when each inner bracket has the same length;
                we assume each dimension has the same meaning; we run transformer directly
                on the inner bracket level, similar to case 2 after embedding table

        dataset: the dataset to train the model. It is used to query certain
            information such as the set of all tokens.
        feature_keys:  list of keys in samples to use as features,
            e.g. ["conditions", "procedures"].
        label_key: key in samples to use as label (e.g., "drugs").
        mode: one of "binary", "multiclass", or "multilabel".
        embedding_dim: the embedding dimension. Default is 128.
        **kwargs: other parameters for the Transformer layer.

    Examples:
        >>> from pyhealth.datasets import SampleEHRDataset
        >>> samples = [
        ...         {
        ...             "patient_id": "patient-0",
        ...             "visit_id": "visit-0",
        ...             "list_codes": ["505800458", "50580045810", "50580045811"],  # NDC
        ...             "list_vectors": [[1.0, 2.55, 3.4], [4.1, 5.5, 6.0]],
        ...             "list_list_codes": [["A05B", "A05C", "A06A"], ["A11D", "A11E"]],  # ATC-4
        ...             "list_list_vectors": [
    """

    def __init__(
        self,
        dataset: SampleEHRDataset,
        feature_keys: List[str],
        label_key: str,
        mode: str,
        pretrained_emb: str = None,
        embedding_dim: int = 128,
        **kwargs
    ):
        super(Transformer, self).__init__(
            dataset=dataset,
            feature_keys=feature_keys,
            label_key=label_key,
            mode=mode,
            pretrained_emb=pretrained_emb,
        )
        self.embedding_dim = embedding_dim

        # validate kwargs for Transformer layer
        if "feature_size" in kwargs:
            raise ValueError("feature_size is determined by embedding_dim")

        # the key of self.feat_tokenizers only contains the code based inputs
        self.feat_tokenizers = {}
        self.label_tokenizer = self.get_label_tokenizer()
        # the key of self.embeddings only contains the code based inputs
        self.embeddings = nn.ModuleDict()
        # the key of self.linear_layers only contains the float/int based inputs
        self.linear_layers = nn.ModuleDict()

        # add feature transformation layers
        for feature_key in self.feature_keys:
            input_info = self.dataset.input_info[feature_key]
            # sanity check
            if input_info["type"] not in [str, float, int]:
                raise ValueError(
                    "Transformer only supports str code, float and int as input types"
                )
            elif (input_info["type"] == str) and (input_info["dim"] not in [2, 3]):
                raise ValueError(
                    "Transformer only supports 2-dim or 3-dim str code as input types"
                )
            elif (input_info["type"] in [float, int]) and (
                input_info["dim"] not in [2, 3]
            ):
                raise ValueError(
                    "Transformer only supports 2-dim or 3-dim float and int as input types"
                )
            # for code based input, we need Type
            # for float/int based input, we need Type, input_dim
            self.add_feature_transform_layer(feature_key, input_info)

        self.transformer = nn.ModuleDict()
        for feature_key in feature_keys:
            self.transformer[feature_key] = TransformerLayer(
                feature_size=embedding_dim, **kwargs
            )
        output_size = self.get_output_size(self.label_tokenizer)
        # transformer's output feature size is still embedding_dim
        self.fc = nn.Linear(len(self.feature_keys) * self.embedding_dim, output_size)

    def forward(self, **kwargs) -> Dict[str, torch.Tensor]:
        """Forward propagation.

        The label `kwargs[self.label_key]` is a list of labels for each patient.

        Args:
            **kwargs: keyword arguments for the model. The keys must contain
                all the feature keys and the label key.

        Returns:
            A dictionary with the following keys:
                loss: a scalar tensor representing the loss.
                y_prob: a tensor representing the predicted probabilities.
                y_true: a tensor representing the true labels.
        """
        patient_emb = []
        for feature_key in self.feature_keys:
            input_info = self.dataset.input_info[feature_key]
            dim_, type_ = input_info["dim"], input_info["type"]

            # for case 1: [code1, code2, code3, ...]
            if (dim_ == 2) and (type_ == str):
                x = self.feat_tokenizers[feature_key].batch_encode_2d(
                    kwargs[feature_key]
                )
                # (patient, event)
                x = torch.tensor(x, dtype=torch.long, device=self.device)
                # (patient, event, embedding_dim)
                x = self.embeddings[feature_key](x)
                # (patient, event)
                mask = torch.any(x !=0, dim=2)

            # for case 2: [[code1, code2], [code3, ...], ...]
            elif (dim_ == 3) and (type_ == str):
                x = self.feat_tokenizers[feature_key].batch_encode_3d(
                    kwargs[feature_key]
                )
                # (patient, visit, event)
                x = torch.tensor(x, dtype=torch.long, device=self.device)
                # (patient, visit, event, embedding_dim)
                x = self.embeddings[feature_key](x)
                # (patient, visit, embedding_dim)
                x = torch.sum(x, dim=2)
                # (patient, visit)
                mask = torch.any(x !=0, dim=2)

            # for case 3: [[1.5, 2.0, 0.0], ...]
            elif (dim_ == 2) and (type_ in [float, int]):
                x, mask = self.padding2d(kwargs[feature_key])
                # (patient, event, values)
                print(len(x),x)
                x = torch.tensor(x, dtype=torch.float, device=self.device)
                
                print(x.shape)

                # (patient, event, embedding_dim)
                x = self.linear_layers[feature_key](x)
                # (patient, event)
                mask = mask.bool().to(self.device)

            # for case 4: [[[1.5, 2.0, 0.0], [1.8, 2.4, 6.0]], ...]
            elif (dim_ == 3) and (type_ in [float, int]):
                x, mask = self.padding3d(kwargs[feature_key])
                # (patient, visit, event, values)
                x = torch.tensor(x, dtype=torch.float, device=self.device)
                # (patient, visit, embedding_dim)
                x = torch.sum(x, dim=2)
                x = self.linear_layers[feature_key](x)
                mask = mask[:, :, 0]
                mask = mask.bool().to(self.device)

            else:
                raise NotImplementedError

            # transform x to (patient, event, embedding_dim)
            if self.pretrained_emb != None:
                x = self.linear_layers[feature_key](x)
        
            _, x = self.transformer[feature_key](x, mask, kwargs.get('register_hook'))
            patient_emb.append(x)
           

        patient_emb = torch.cat(patient_emb, dim=1)

        logits = self.fc(patient_emb)
        return logits

        
        # obtain y_true, loss, y_prob
        y_true = self.prepare_labels(kwargs[self.label_key], self.label_tokenizer)
        loss = self.get_loss_function()(logits, y_true)
        y_prob = self.prepare_y_prob(logits)
        results = {"loss": loss, "y_prob": y_prob, "y_true": y_true, "logit": logits}
        if kwargs.get("embed", False):
            results["embed"] = patient_emb
        
        return results